# Scheduled Severless Startup
> Using AWS lambda to automate boot up and down of a Digital Ocean droplet with Slack notifications

- toc: true 
- badges: true
- comments: true
- categories: [aws lambda, automation, python]
- image: images/chart-preview.png
- show_image:true
- hide:true

# The Problem

As part of the Covid-19 effort, Digital Ocean donated some free credit to us to work on a local food delivery scheme.

To make that credit go as far as possible and to minimise power consumption, we'd like to power up and down the servers according to a schedule.


# The Solution

Esimated cost saving...

Here's how to do that with AWS Lambda, with cloudfront events. We iterate on that to use the Serverless Framework.



## Prerequisites

To follow along, there is a bit of set up involved to launch a Droplet that you can inspect with SSH if you need. 

You could always just check the [companion repo.](https://github.com/jonwhittlestone/scheduled-serverless-startup)

You will need to:


* .. have `jq` installed to format JSON responses
    * [Download JQ](https://stedolan.github.io/jq/download/) for your OS
    
    
* .. [generate](https://www.digitalocean.com/docs/apis-clis/api/) an access token so you can use the Digital Ocean API.


* .. add the access token to your environment

    ```
    $ export DIGITAL_OCEAN_ACCESS_TOKEN=[your_digital_ocean_token]
    ```
    
* .. have a [running Digital Ocean Droplet](https://www.digitalocean.com/docs/droplets/quickstart/#create-droplets) with Docker installed. 

    * If you need to create one, use the following [API operation](https://developers.digitalocean.com/documentation/v2/#create-a-new-droplet).
        ```
        $ curl -X POST \
        -H 'Content-Type: application/json' \
        -H 'Authorization: Bearer '$DIGITAL_OCEAN_ACCESS_TOKEN'' \
        -d '{"name":"scheduled-serverless","region":"lon1","size":"s-2vcpu-4gb","image":"docker-18-04"}' \
        "https://api.digitalocean.com/v2/droplets" 
        ```
    
    * .. [generate](https://8gwifi.org/sshfunctions.jsp) and add the private key to your droplet. When generating, Select: `RSA` and `4096` and leave `Passphrase` blank. You may use the following API operation.

    * .. add the public key to your environment



## Shutdown a Droplet with CURL

Assuming, your account only has one Digital Ocean droplet, the following CURL statement shows `active` status and the ID

    $ curl \
        -H 'Content-Type: application/json' \
        -H 'Authorization: Bearer '$DIGITAL_OCEAN_ACCESS_TOKEN'' \
        "https://api.digitalocean.com/v2/droplets?name=scheduled-serverless" | jq '.droplets[0].status, .droplets[1].id'
      % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                     Dload  Upload   Total   Spent    Left  Speed
    100  4137    0  4137    0     0   7925      0 --:--:-- --:--:-- --:--:--  7940
    "active"
    194534673



Issuing the following [CURL will](https://developers.digitalocean.com/documentation/v2/#shutdown-a-droplet) shut it down.

    curl -X POST \
    -H 'Content-Type: application/json' \
    -H 'Authorization: Bearer '$DIGITAL_OCEAN_ACCESS_TOKEN'' \
    -d '{"type":"shutdown"}' \
    "https://api.digitalocean.com/v2/droplets/194534673/actions"


We can then verify  the droplet has been powered down by reissuing the previous command from above. to show the 'active' status.
    
    ...
    "off"
    194534673



## Power on a Droplet with CURL

The action for powering on follows the same convention.

    $ curl -X POST \
        -H 'Content-Type: application/json' \
        -H 'Authorization: Bearer '$DIGITAL_OCEAN_ACCESS_TOKEN'' \
        -d '{"type":"power_on"}' \
        "https://api.digitalocean.com/v2/droplets/194534673/actions"

We can check the machine has been powered down:

    $ curl \
            -H 'Content-Type: application/json' \
            -H 'Authorization: Bearer '$DIGITAL_OCEAN_ACCESS_TOKEN'' \
        "https://api.digitalocean.com/v2/droplets?name=scheduled-serverless" | jq '.droplets[] | {id:.id, name:.name,status: .status}'


## Requests scripts in AWS Lambda

First lets translate these CURL statements into Python.

In [2]:
import requests

headers = {
    'Content-Type': 'application/json',
#     'Authorization': 'Bearer $DIGITAL_OCEAN_ACCESS_TOKEN',
    'Authorization': 'Bearer XXX',
}

params = (
    ('page', '1'),
    ('per_page', '100'),
)

response = requests.get('https://api.digitalocean.com/v2/droplets', headers=headers, params=params)


 
#data = '{"type":"shutdown"}'

#response = requests.post('https://api.digitalocean.com/v2/droplets/194534673/actions', headers=headers, data=data)
print(response)

<Response [401]>


## Schedule the scripts with Cloudwatch events

## Serverless framework

## Team notifications with Slack

As part of what's become known as 'Chatops', it's useful to update team members and stakeholders about the status of the server.

There are a variety of messaging platforms that have APIs such as Telegram or even SMS, but Slack has become one of the most popular.